In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from tpot import TPOTClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [ ]:
label_encoder = LabelEncoder()
scaler = StandardScaler()
column_names = [f'feature_{x}' for x in range(75)]

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
submission_sample = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')
test = test.drop(['id'],axis=1)
X=train.drop(['target','id'],axis=1)
Y=train['target']
Y=label_encoder.fit_transform(Y)

In [ ]:
X[column_names] = scaler.fit_transform(X[column_names])
test[column_names] = scaler.transform(test[column_names])

In [ ]:
%%time 

X_train, X_test, y_train, y_test  = train_test_split(X,Y,train_size=0.8,random_state=42)
# TPOT setup
GENERATIONS = 3
POP_SIZE = 100
CV = 5
SEED = 12

tpot = TPOTClassifier(
    generations=GENERATIONS,
    population_size=POP_SIZE,
    random_state=SEED,
    config_dict="TPOT cuML",
    n_jobs=1,
    cv=CV,
    verbosity=2,
    scoring='neg_log_loss'
)

tpot.fit(X_train, y_train)

preds = tpot.predict_proba(X_test)
print(log_loss(y_test, preds))

In [ ]:
test_preds = tpot.predict_proba(test)
test_preds

In [ ]:
submission = pd.DataFrame(test_preds,columns=[f'Class_{x+1}' for x in range(9)])
submission.insert(0,'id',submission_sample['id'])
submission.head(5)

In [ ]:
submission.to_csv('tpot_rapids.csv',index=False)